In [35]:
# API_KEY=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
import groq  # assuming groq is the SDK or library for using the Groq API
import json
import numpy as np
import requests
import os

import pandas as pd
from groq import Groq


In [36]:
client = Groq(api_key='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')


In [37]:
verbs = pd.read_csv('/Users/malhar.inamdar/Desktop/test/hindi-verbs.txt', header=None)
adj = pd.read_csv('/Users/malhar.inamdar/Desktop/test/adjectives_hindi.txt', header=None)
nouns = pd.read_csv('/Users/malhar.inamdar/Desktop/test/nouns_hindi.txt', header=None)
end = np.array(["happy", "sad", "mysterious"])


In [46]:
def generatePrompt():
    verb = verbs.sample(n=1).iloc[0, 0]
    noun = nouns.sample(n=1).iloc[0, 0]
    adjective = adj.sample(n=1).iloc[0, 0]

    ending = np.random.choice(end)
    word_choice = np.random.choice(["verb", "noun", "adjective"])
    word = verb if word_choice == "verb" else noun if word_choice == "noun" else adjective

    basePromptHindi = f'''Write a simple Hindi children's story for a 5-year-old in 4-5 lines. The story should be inspired by Panchtantra tales and use easy, understandable language. Make it engaging with a clear beginning, middle, and end, and include only one brief dialogue between the characters. The story should have a gentle, natural resolution that reflects an emotion, such as happiness, friendship, or kindness, without explicitly using these words. Use the {word_choice} "{word}" naturally within the story.
    The story should naturally lead to a {ending} ending, conveyed through the characters’ reactions, atmosphere, or conclusion without directly using the word {ending}.
    Just give the Hindi story.'''
    
    return {'instruction': basePromptHindi, "word": word, "ending": ending}



In [49]:
def store_dataset(n):
    data = {
        "output": [],
        "input": [],
        "instruction": []
    }
    
    for _ in range(n):
        prompt_data = generatePrompt()
        instruction = prompt_data['instruction']
        
        # Make API call to generate story using chat completion
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": instruction
                }
            ],
            model="llama3-70b-8192",
            max_tokens=680
        )
        
        # Extract story text from response
        story = response.choices[0].message.content.strip()
        
        # Store data
        data["input"].append(instruction)
        data["output"].append(story)
        data["instruction"].append({"word": prompt_data["word"], "ending": prompt_data["ending"]})
    
    # Save to JSON file
    with open('hindi_stories_dataset3.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [50]:
store_dataset(10)
